In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt


In [2]:
cust=pd.read_csv(r'C:\Users\Sai\Desktop\notes\sowmiya\Python\Python Foundation Case Study 3 - Insurance Claims Case Study\cust_demographics.csv')

In [3]:
claims = pd.read_csv(r'C:\Users\Sai\Desktop\notes\sowmiya\Python\Python Foundation Case Study 3 - Insurance Claims Case Study\claims.csv')

In [4]:
data=pd.merge(left=cust,right=claims,left_on="CUST_ID",right_on="customer_id",how="left")

In [5]:
data.drop(columns="CUST_ID",inplace=True)

# 2. Perform a data audit for the datatypes and find out if there are any  mismatch within the current datatypes of the columns and their business significance

In [6]:
data.dtypes

gender                  object
DateOfBirth             object
State                   object
Contact                 object
Segment                 object
claim_id               float64
customer_id            float64
incident_cause          object
claim_date              object
claim_area              object
police_report           object
claim_type              object
claim_amount            object
total_policy_claims    float64
fraudulent              object
dtype: object

In [7]:
#dateof birth , claim_date , claim_amount should be of date and float

data.DateOfBirth=pd.to_datetime(data.DateOfBirth,format='%d-%b-%y')
data.claim_date=pd.to_datetime(data.claim_date,format='%m/%d/%Y')



# Convert the column claim_amount to numeric. Use the appropriate modules/attributes to remove the $ sign

In [8]:
data.claim_amount=np.float32(data.claim_amount.str[1:])

# 4.Of all the injury claims, some of them have gone unreported with the police. Create an alert flag (1,0) for all such claims.


In [9]:
data['alert_flag']=np.where(data.police_report == 'Yes',0,1)

# 5.One customer can claim for insurance more than once and in each claim,multiple categories of claims can be involved. However, customer ID should remain unique. 

In [10]:
#Retain the most recent observation and delete any duplicated records in
#the data based on the customer ID column

In [11]:
data.drop_duplicates(keep='last',subset='customer_id',inplace=True)

In [12]:
data.reset_index(inplace=True)

# 6 Check for missing values and impute the missing values with an appropriate value. (mean for continuous and mode for categorical

In [14]:
categorical=data.select_dtypes('object').columns
[data[i].fillna(data[i].mode()[0],inplace=True) for i in categorical]

[None, None, None, None, None, None, None, None, None]

In [15]:
countinous = data.select_dtypes('number').columns
countinous

Index(['index', 'claim_id', 'customer_id', 'claim_amount',
       'total_policy_claims', 'alert_flag'],
      dtype='object')

In [16]:
#claim_id,customer_id is unique so droped the fields which is not having those
data.dropna(subset=['customer_id','claim_id'],inplace=True)

In [17]:
[data[i].fillna(data[i].mean(),inplace=True) for i in countinous]

[None, None, None, None, None, None]

In [18]:
data.isna().sum()

index                  0
gender                 0
DateOfBirth            0
State                  0
Contact                0
Segment                0
claim_id               0
customer_id            0
incident_cause         0
claim_date             0
claim_area             0
police_report          0
claim_type             0
claim_amount           0
total_policy_claims    0
fraudulent             0
alert_flag             0
dtype: int64

# 7. Calculate the age of customers in years. Based on the age, categorize thecustomers according to the below criteria
Children < 18
Youth 18-30
Adult 30-60
Senior > 60

In [19]:
data['age']=data.DateOfBirth.dt.year-dt.datetime.now().year

In [20]:
data['age']=abs(data['age'])

In [21]:
data['AgeGroup']=np.where(((data['age']<18) & (data['age']>0)),'Youth',np.where(((data['age']>=18) & (data['age']<=30)),'Adult','Senior') )

# 8. What is the average amount claimed by the customers from various segments?

In [22]:
data.pivot_table(index='Segment',values='claim_amount')

,claim_amount
Segment,
Gold,12846.386719
Platinum,12371.645508
Silver,12267.146484


# 9. What is the total claim amount based on incident cause for all the claimsthat have been done at least 20 days prior to 1st of October, 2018.


In [53]:
data.sample()

least_date=dt.datetime.strptime("2018-09-01",'%Y-%m-%d')-dt.timedelta(days=20)
least_date
data[data['claim_date'] < least_date].groupby('incident_cause')['claim_amount'].sum().sum()


11913022.0

# 10. How many adults from TX, DE and AK claimed insurance for driver related issues and causes?

In [ ]:
len(data.loc[((data.State =='TX')|(data.State =='DE')|(data.State =='AK'))& (data.incident_cause.str.lower().str.contains('driver')) & (data.AgeGroup=='Adult')])

# 11. Draw a pie chart between the aggregated value of claim amount based on gender and segment. Represent the claim amount as a percentage onthe pie chart.

In [ ]:
data.pivot_table(index='Segment',values='claim_amount',columns='gender').plot(kind='pie',subplots=True, autopct='%.1f%%',legend=False)

# 12 .Among males and females, which gender had claimed the most for any type of driver related issues? E.g. This metric can be compared using a bar chart

In [ ]:
driver_issues=data.loc[data['incident_cause'].str.lower().str.contains('driver')]

In [ ]:
driver_issues.pivot_table(index='gender',values='incident_cause',aggfunc='count').plot(kind='bar')

# 13. Which age group had the maximum fraudulent policy claims? Visualize it on a bar chart.

In [ ]:
data[data['fraudulent']=='Yes'].groupby('AgeGroup')['AgeGroup'].count().plot(kind='barh')

# 14.Visualize the monthly trend of the total amount that has been claimed by the customers. Ensure that on the “month” axis, the month is in a chronological order not alphabetical order.

In [ ]:
data['claim_month']=data['claim_date'].dt.month

In [ ]:
data.pivot_table(index='claim_month',values='claim_amount',aggfunc='sum').plot(kind='line')

# 15.What is the average claim amount for gender and age categories and suitably represent the above using a facetted bar chart, one facet that represents fraudulent claims and the other for non-fraudulent claims.

In [ ]:
data.pivot_table(index='gender',columns='fraudulent',values='claim_amount').plot(kind='bar')

Based on the conclusions from exploratory analysis as well as suitable 
statistical tests, answer the below questions. Please include a detailed 
write-up on the parameters taken into consideration, the Hypothesis 
testing steps, conclusion from the p-values and the business implications of 
the statements.

# 16.Is there any similarity in the amount claimed by males and females

In [ ]:
import scipy.stats as st


In [ ]:
# Two sample t-test:
# Data values must be independent. Measurements for one observation do not affect measurements for any other observation.
# Data in each group must be obtained via a random sample from the population.
# Data in each group are normally distributed.
# Data values are continuous.
# The variances for the two independent groups are equal.
import scipy.stats as st
female = df['claim_amount'].loc[df['gender']=="Female"]
male = df['claim_amount'].loc[df['gender']=="Male"]
st.ttest_ind(female, male, equal_var=False)
# As the significance value of t test is greater than 0.05 we can safely conclude that there is a similarity in amount claimed by males and females.

# 17.Is there any relationship between age category and segment

In [ ]:
# 17. Is there any relationship between age category and segment?
# Chi Square test:
# H0 = No relation between category and segment
# Ha = There is a relationship between category and segment
c = pd.crosstab(data.AgeGroup, data.Segment, margins = True)
st.chi2_contingency(observed= c)
# Since the significance value of the test is greter than 0.05, we fail reject the null hypothesis. Therefore there is no relationship between age and segment

In [ ]:
# 18. The current year has shown a significant rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.
# Correlation
data['year'] = pd.DatetimeIndex(data.claim_date).year
CY = data.loc[data.year == 2018]["claim_amount"]
PY = data.loc[data.year == 2017]["claim_amount"]
CY.corr(other= PY)
# There is no correlation between Current Year and Previous year

In [ ]:
# 19. Is there any difference between age groups and insurance claims?
# ANOVA:
# H0 = No relation between category and segment
# Ha = There is a relationship between category and segment
a1 = data['total_policy_claims'].loc[data['AgeGroup']=="Senior"]
a2 = data['total_policy_claims'].loc[data['AgeGroup']=="Adult"]
st.f_oneway(a1,a2)
# Since the significance value of the test is greater than 0.05, we fail reject the null hypothesis.there is no difference between age group and insurance claims

In [ ]:
# 20. Is there any relationship between total number of policy claims and the claimed amount?
# Correlation
data.total_policy_claims.corr(other= data.claim_amount)
# As the correlation is negative the number of policy claims in inversely propotional to the claimed amount.